In [ ]:

import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from astropy.table import Table
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
import matplotlib.pyplot as plt
from os import path, makedirs
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
from calibpipe.throughput import MuonCalibrator
from ctapipe.io import SimTelEventSource
from ctapipe.io import HDF5TableWriter
from ctapipe.image import ImageExtractor
from ctapipe.calib import CameraCalibrator
from ctapipe.instrument import CameraGeometry
from ctapipe.visualization import CameraDisplay
from ctapipe.image import TailcutsImageCleaner
from ctapipe.instrument import TelescopeDescription, SubarrayDescription
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from ctapipe.image.muon import ring_containment, ring_completeness, intensity_ratio_inside_ring, mean_squared_error
from ctapipe.containers import MuonParametersContainer

from ctapipe.containers import MuonHoughContainer

from ctapipe.coordinates import TelescopeFrame, CameraFrame
from astropy.coordinates import SkyCoord

#%matplotlib inline
from tqdm import tqdm
import numpy as np
import glob
from os.path import basename
from sys import argv
#import mpld3
import matplotlib.pyplot as plt
import astropy.units as u

#Full array print
#import sys
#np.set_printoptions(threshold=sys.maxsize)
from joblib import Parallel, delayed

def HoughT(x, y, z, d, w, f):
# *** HoughT retorna aquell anell que millor s'ajusta a l'anell generat pel muo, donant el centre i coordenades x-y d'aquest
# *** Variables:
#    x són les coordenades en l'eix X de l'anell del muo
#    y són les coordenades en l'eix Y de l'anell del muo
#    z és la intensitat en cada punt de l'anell del muo
#    w és el gruix de l'anell que s'ajusta
#    f és la focal utilitzada per passar de mestres a graus
    x = x.value
    y = y.value
    R_min = 0.9
    R_max = 1.4
    R = np.linspace(R_min, R_max, 50)*f/180.*np.pi
    A = np.zeros((len(x),len(R),5))

    for j in range(len(R)):
        for i in range(len(x)):
            Range_0 = np.sqrt((x - x[i]) ** 2 + (y - y[i]) ** 2)
            Range_1 = np.sqrt((x - (x[i] + d)) ** 2 + (y - y[i]) ** 2)
            Range_2 = np.sqrt((x - (x[i] - d)) ** 2 + (y - y[i]) ** 2)
            Range_3 = np.sqrt((x - x[i]) ** 2 + (y - (y[i] + d)) ** 2)
            Range_4 = np.sqrt((x - x[i]) ** 2 + (y - (y[i] - d)) ** 2)

            Q_0 = (z)[(Range_0 <= R[j]) & (Range_0 > R[j]-w)]
            Q_1 = (z)[(Range_1 <= R[j]) & (Range_1 > R[j] - w)]
            Q_2 = (z)[(Range_2 <= R[j]) & (Range_2 > R[j] - w)]
            Q_3 = (z)[(Range_3 <= R[j]) & (Range_3 > R[j] - w)]
            Q_4 = (z)[(Range_4 <= R[j]) & (Range_4 > R[j] - w)]

            A[i][j][0] = np.sum(Q_0) / len(Q_0)
            A[i][j][1] = np.sum(Q_1) / len(Q_1)
            A[i][j][2] = np.sum(Q_2) / len(Q_2)
            A[i][j][3] = np.sum(Q_3) / len(Q_3)
            A[i][j][4] = np.sum(Q_4) / len(Q_4)

    #mA = np.argmax(A)
    mA = np.unravel_index(A.argmax(), A.shape)

    if mA[2] == 0:
        mpix = np.sqrt((x - x[mA[0]]) ** 2 + (y - y[mA[0]]) ** 2)
        center_x = x[mA[0]]
        center_y = y[mA[0]]
    if mA[2] == 1:
        mpix = np.sqrt((x - (x[mA[0]] + d)) ** 2 + (y - y[mA[0]]) ** 2)
        center_x = x[mA[0]] + d
        center_y = y[mA[0]]
    if mA[2] == 2:
        mpix = np.sqrt((x - (x[mA[0]] - d)) ** 2 + (y - y[mA[0]]) ** 2)
        center_x = x[mA[0]] - d
        center_y = y[mA[0]]
    if mA[2] == 3:
        mpix = np.sqrt((x - x[mA[0]]) ** 2 + (y - (y[mA[0]] + d)) ** 2)
        center_x = x[mA[0]]
        center_y = y[mA[0]] + d
    if mA[2] == 4:
        mpix = np.sqrt((x - x[mA[0]]) ** 2 + (y - (y[mA[0]] - d)) ** 2)
        center_x = x[mA[0]]
        center_y = y[mA[0]] - d

    circle_x = x[(mpix <= R[mA[1]]) & (mpix > R[mA[1]]-w)]
    circle_y = y[(mpix <= R[mA[1]]) & (mpix > R[mA[1]]-w)]
    radius = R[mA[1]]
    return center_x, center_y, circle_x, circle_y, radius, w



In [ ]:
cherenkov_angle = 1.2
cherenkov_angle_tan = math.tan(np.deg2rad(cherenkov_angle))
lapalma_alt = 2180
paranal_alt = 2150
lst_alt = 16
mst_alt = 9
sst_alt = 3.25
lst_mirror = 11.5
mst_mirror = 6.25
sst_mirror = 2.15


### Calculating fixchi for each telescope

In [ ]:
def fixchi(altitude, mirror_radius):
    return 1.3 * (altitude + ((2*mirror_radius)/cherenkov_angle_tan))


In [ ]:
fixchi(lapalma_alt + sst_alt, sst_mirror)

In [ ]:
np.deg2rad(90)

### Atmosphere interpolation

In [ ]:
atmospheric_thickness = [1036, 920, 818, 726, 643, 568, 499, 438, 383, 333, 288, 249, 213, 182, 155, 132, 112, 95, 80, 67, 57]
altitude = [i for i in range(21)]
xvals = np.linspace(0, 20, 20000)

interpolated_thickness = np.interp(xvals, altitude, atmospheric_thickness)

#plt.scatter(x,y, s = 200)
plt.scatter(xvals, interpolated_thickness, alpha = 0.2)
plt.grid()

In [ ]:
interpolated_thickness[4300]

### Processing simtel files

In [ ]:
lst_file = "/Users/vdk/GeneveWork/Muons/data/run401_muon_lst.simtel.gz"
mst_nectar_file = "/Users/vdk/GeneveWork/Muons/data/run101_muon_mst_nectar.simtel.gz"
mst_flash_file = "/Users/vdk/GeneveWork/Muons/data/run101_muon_mst_flash.simtel.gz"
sst_file = "/Users/vdk/GeneveWork/Muons/data/sst/run101_muon_sst.simtel.gz"

int_cut = 0.2
min_pixel = 50
muon_processor_config = Config({
                                "MuonProcessor": {
                                    "RingQuery": {
                                        "quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.1"],
                                                        ["ring_completeness", "parameters.completeness > 0.1"]]},
                                    "ImageParameterQuery" :{
                                        "quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})


In [ ]:
tel_id = 1
source = SimTelEventSource(mst_flash_file, max_events = 100000)
telescope = source.subarray.tel[tel_id]
cam = telescope.camera.geometry
cam.cam_rotation
telescope.optics.equivalent_focal_length

In [ ]:
number_of_events = []

i = 0
source = EventSource(sst_file, max_events = 100000)
event_iterator = iter(source)
for i,event in enumerate(event_iterator):
    i+=1
number_of_events.append(i)
print(f"Number of events in SST = {i}")

i=0    
source = EventSource(mst_flash_file, max_events = 100000)
event_iterator = iter(source)
for i,event in enumerate(event_iterator):
    i+=1
number_of_events.append(i)
print(f"Number of events in MST FlashCam = {i}")

i=0 
source = EventSource(mst_nectar_file, max_events = 100000)
event_iterator = iter(source)
for i,event in enumerate(event_iterator):
    i+=1
number_of_events.append(i)
print(f"Number of events in MST NectarCam = {i}")

i=0 
source = EventSource(lst_file, max_events = 100000)
event_iterator = iter(source)
for i,event in enumerate(event_iterator):
    i+=1
number_of_events.append(i)
print(f"Number of events in LST = {i}")



In [ ]:
int_cut = 1
min_pixel = 20
muon_processor_config = Config({
                                "MuonProcessor": {
                                    "RingQuery": {
                                        "quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.1"],
                                                        ["ring_completeness", "parameters.completeness > 0.1"]]},
                                    "ImageParameterQuery" :{
                                        "quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})
event_ids_calib = []
impacts = []
widths = []
source = EventSource(mst_flash_file, max_events = 100000)
event_iterator = iter(source)
image_processor = ImageProcessor(source.subarray)
muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
for i,event in enumerate(event_iterator):

    calib(event)
    image_processor(event)
    muon_processor(event)
    
    if not np.isnan(event.muon.tel[1].efficiency.width):
    #if True:
        camgeom = source.subarray.tel[1].camera.geometry
        title = f'efficiency = {round(event.muon.tel[1].efficiency.optical_efficiency, 3)}'
        disp = CameraDisplay(camgeom,title=title)
        disp.image = event.dl1.tel[1].image
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        plt.show()
        plt.close() 

### SST PROCESSING

In [ ]:
single_file = '/Users/vdk/GeneveWork/Muons/data/sst/run101_muon_sst.simtel.gz'
print('Now treating',single_file)

source = SimTelEventSource( input_url=single_file)
event_iterator = iter(source)
event = next(event_iterator)

In [ ]:
event = next(event_iterator)

In [ ]:

print('Source', source, source.subarray)

# writer = HDF5TableWriter('/Users/vdk/GeneveWork/Muons/data/sst/output2.hdf5', "muons", mode="w")
# muon_table_name = 'muon_table'
# writer.exclude(muon_table_name, 'prediction')
# writer.exclude(muon_table_name, 'mask')

extractor_name = 'GlobalPeakWindowSum'
extractor = ImageExtractor.from_name(extractor_name,subarray=source.subarray)

calib = CameraCalibrator(subarray=source.subarray, image_extractor=extractor)

picture_threshold_pe = 7.
boundary_threshold_pe = 3.
cleaning = TailcutsImageCleaner(subarray=source.subarray,
                                picture_threshold_pe=picture_threshold_pe, boundary_threshold_pe=boundary_threshold_pe)

min_pixels = 8
pedestal_noise_rms = 1.05 

ring_fitter = MuonRingFitter(fit_method="taubin")
#ring_fitter = MuonRingFitter(fit_method="kundu_chaudhuri")
intensity_fitter = MuonIntensityFitter(subarray=source.subarray)

max_counter = -1  # for testing, set it to a small positive number
cnt = 0
tel_id = 1  # Here, because we have simulated only one telescope!



event_id = event.index.event_id

calib(event)

image = event.dl1.tel[tel_id].image
print ('event: ', event.dl1.tel[tel_id])
print ('Image Length', len(image))

clean_mask = cleaning(tel_id, image)

telescope = source.subarray.tel[tel_id]
cam = telescope.camera.geometry

camera_frame = CameraFrame(focal_length=telescope.optics.equivalent_focal_length,
                            rotation=cam.cam_rotation)
cam_coords = SkyCoord(x=cam.pix_x, y=cam.pix_y, frame=camera_frame)
tel_coord = cam_coords.transform_to(TelescopeFrame())

x, y = tel_coord.fov_lon, tel_coord.fov_lat
print ('x,y length:', len(cam.pix_x),len(cam.pix_y))

camgeom = source.subarray.tel[1].camera.geometry
title = f'i'
disp = CameraDisplay(camgeom,title=title)
#disp.image = event.r1.tel[1].waveform.sum(axis = 1)
disp.image = event.r1.tel[1].waveform.sum(axis=1)
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)
plt.show()
plt.close() 

mask = clean_mask
for i in range(3):
    ring = ring_fitter(x, y, image, mask)
    dist = np.sqrt((x - ring.center_fov_lon)**2 + (y - ring.center_fov_lat)**2)
    mask = np.abs(dist - ring.radius) / ring.radius < 0.4
    
H_pixel_width = CameraGeometry.guess_pixel_width(cam.pix_x.value, cam.pix_y.value)
H_ring = HoughT(cam.pix_x, cam.pix_y, disp.image, H_pixel_width/2., 0.01, telescope.optics.equivalent_focal_length.value)
H_radius  = H_ring[4] - H_pixel_width/2.
dist = np.sqrt((cam.pix_x.value - H_ring[0]) ** 2 + (cam.pix_y.value - H_ring[1]) ** 2)
H_mask = np.abs(dist - H_radius) / H_radius < 0.4

In [ ]:
border = cam.get_border_pixel_mask()
fov_radius = np.sqrt(x[border]**2 + y[border]**2).mean()
containment = ring_containment(
ring.radius,
ring.center_fov_lon,
ring.center_fov_lat,
fov_radius,
)

H_fov_radius = np.sqrt(cam.pix_x.value[border] ** 2 + cam.pix_y.value[border] ** 2).mean()
H_containment = ring_containment(
H_radius,
H_ring[0],
H_ring[1],
H_fov_radius,
)

completeness_threshold = 25   #  defaults
completeness_bins = 16        #  defaults
completeness = ring_completeness(
x[mask], y[mask], image[mask],
ring.radius, ring.center_fov_lon, ring.center_fov_lat,
completeness_threshold,  completeness_bins
)

H_completeness = ring_completeness(
cam.pix_x.value[H_mask], cam.pix_y.value[H_mask], image[H_mask],
H_radius, H_ring[0], H_ring[1],
completeness_threshold, completeness_bins
)

pixel_width = CameraGeometry.guess_pixel_width(x,y)
#print("witdh ", pixel_width)
#wait=input()
ratio_width = 1.5             # defaults
intensity_ratio = intensity_ratio_inside_ring(
x[clean_mask], y[clean_mask],
image[clean_mask],
ring.radius, ring.center_fov_lon, ring.center_fov_lat,
width=ratio_width * pixel_width,
)

H_intensity_ratio = intensity_ratio_inside_ring(
cam.pix_x.value[clean_mask], cam.pix_y.value[clean_mask],
image[clean_mask],
H_radius, H_ring[0], H_ring[1],
width=ratio_width * H_pixel_width,
)

mse = mean_squared_error(
x[clean_mask], y[clean_mask], image[clean_mask],
ring.radius, ring.center_fov_lon, ring.center_fov_lat
)

H_mse = mean_squared_error(
cam.pix_x.value[clean_mask], cam.pix_y.value[clean_mask], image[clean_mask],
H_radius, H_ring[0], H_ring[1]
)

params = MuonParametersContainer(
    containment=containment,
    completeness=completeness,
    intensity_ratio=intensity_ratio,
    mean_squared_error=mse
)

# intensity_fitter does not support a mask yet, set ignored pixels to 0
H_image = image
image[~mask] = 0
H_image[~H_mask] = 0
intens = intensity_fitter(tel_id,ring.center_fov_lon,ring.center_fov_lat,ring.radius,
                        image,pedestal=pedestal_noise_rms)

cam_coord_H = SkyCoord(x=H_ring[0] * u.m, y=H_ring[1] * u.m, frame=camera_frame)
tel_coord_H = cam_coord_H.transform_to(TelescopeFrame())
H_intens = intensity_fitter(tel_id, tel_coord_H.fov_lon.deg*u.deg, tel_coord_H.fov_lat.deg*u.deg, H_radius*u.deg/telescope.optics.equivalent_focal_length.value*180./np.pi,
                            H_image, pedestal=pedestal_noise_rms)

hough = MuonHoughContainer(
    H_center_x = tel_coord_H.fov_lon.deg,
    H_center_y = tel_coord_H.fov_lat.deg,
    H_radius = H_radius * u.m / telescope.optics.equivalent_focal_length * 180. / np.pi,
    H_width = H_ring[5] * u.m / telescope.optics.equivalent_focal_length * 180. / np.pi,
    H_containment = H_containment,
    H_completeness = H_completeness,
    H_intensity_ratio = H_intensity_ratio,
    H_mean_squared_error = H_mse,
    H_width_intens = H_intens['width'],
    H_impact = H_intens['impact'],
    H_impact_x = H_intens['impact_x'],
    H_impact_y = H_intens['impact_y'],
    H_optical_efficiency = H_intens['optical_efficiency'],
    H_impact_error = H_intens['impact_error'],
    H_phi_error = H_intens['phi_error'],
    H_width_intens_error = H_intens['width_error'],
    H_optical_efficiency_error = H_intens['optical_efficiency_error']
)
print(f'Muon fit: r={ring.radius:.2f}'
        f', width={intens.width:.4f}'
        f', efficiency={intens.optical_efficiency:.2%}',
        f', mse={params.mean_squared_error:.2f}',
        f', completeness={params.completeness:.2%}',
        f', r={hough.H_radius:.2f}',
        f', completeness={hough.H_completeness:.2%}'
)

In [ ]:
intens.is_valid

In [ ]:
hough.H_optical_efficiency

In [ ]:
source = EventSource(mst_flash_file, max_events = 20)
event_iterator = iter(source)

for i,event in enumerate(event_iterator):
    camgeom = source.subarray.tel[1].camera.geometry
    title = f'FlashCam muon ring {event.index.event_id}'
    disp = CameraDisplay(camgeom,title=title)
    disp.image = event.r1.tel[1].waveform.sum(axis = 1)
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    # bx = axes[1]
    # for pix_id in range(0,1855):
    #     #bx.plot(j.r0.tel[teln].waveform[0][pix_id], label="pix {}".format(pix_id), drawstyle='steps')
    #     bx.plot(waveform[pix_id], label="pix {}".format(pix_id), drawstyle='steps')
    plt.show()
    plt.close()        

In [ ]:
source = EventSource(mst_nectar_file, max_events = 20)
event_iterator = iter(source)

for i,event in enumerate(event_iterator):
    camgeom = source.subarray.tel[1].camera.geometry
    title = f'NectarCam muon ring {event.index.event_id}'
    disp = CameraDisplay(camgeom,title=title)
    disp.image = event.r1.tel[1].waveform.sum(axis = 1)
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    # bx = axes[1]
    # for pix_id in range(0,1855):
    #     #bx.plot(j.r0.tel[teln].waveform[0][pix_id], label="pix {}".format(pix_id), drawstyle='steps')
    #     bx.plot(waveform[pix_id], label="pix {}".format(pix_id), drawstyle='steps')
    plt.show()
    plt.close()        

### SST Muon Rings

In [ ]:
source = EventSource(sst_file, max_events = 40)
event_iterator = iter(source)

for event in event_iterator:
    camgeom = source.subarray.tel[1].camera.geometry
    title = f'SST ASTRI muon ring {event.index.event_id}'
    disp = CameraDisplay(camgeom,title=title)
    disp.image = event.r1.tel[1].waveform.sum(axis = 1)
    #disp.image = event.simulation.tel[1].true_image
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    # bx = axes[1]
    # for pix_id in range(0,1855):
    #     #bx.plot(j.r0.tel[teln].waveform[0][pix_id], label="pix {}".format(pix_id), drawstyle='steps')
    #     bx.plot(waveform[pix_id], label="pix {}".format(pix_id), drawstyle='steps')
    plt.show()
    plt.close()        

In [ ]:
source.subarray.tel[1].optics.n_mirrors

### LST Muon Rings

In [ ]:
source = EventSource(lst_file, max_events = 100)
event_iterator = iter(source)

for event in event_iterator:
    camgeom = source.subarray.tel[1].camera.geometry
    title = f'LST muon ring {event.index.event_id}'
    disp = CameraDisplay(camgeom,title=title)
    disp.image = event.r1.tel[1].waveform.sum(axis = 1)
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    # bx = axes[1]
    # for pix_id in range(0,1855):
    #     #bx.plot(j.r0.tel[teln].waveform[0][pix_id], label="pix {}".format(pix_id), drawstyle='steps')
    #     bx.plot(waveform[pix_id], label="pix {}".format(pix_id), drawstyle='steps')
    plt.show()
    plt.close()        

### Fitting Muons

In [ ]:
source = EventSource(mst_nectar_file, max_events = 10000)
event_iterator = iter(source)

image_processor = ImageProcessor(source.subarray)
muon_processor = MuonProcessor(source.subarray, config = Config(muon_processor_config))
calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray, config = Config(muon_processor_config))

for i,event in enumerate(event_iterator):
    calib(event)
    image_processor(event)
    muon_processor(event)
    if not np.isnan(event.muon.tel[1].efficiency.optical_efficiency):
        print(event.muon.tel[1].efficiency.optical_efficiency)
        camgeom = source.subarray.tel[1].camera.geometry
        title = f'i'
        disp = CameraDisplay(camgeom,title=title)
        #disp.image = event.r1.tel[1].waveform.sum(axis = 1)
        disp.image = event.dl1.tel[1].image
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        plt.show()
        plt.close() 
            

In [ ]:
event.simulation.shower

### M. Gaug muon processing

In [ ]:
import tables as tb
import glob
import numpy as np
from astropy import units as u
from os.path import basename

muonFile="run101_muon_mst_flash.simtel.hdf5"
#muonFile="/muon_0deg_354.681deg_run53___cta-prod4-sst-astri_desert-2180m-LaPalma-sst-astri.simtel.hdf5"
#muonFile="/muon_30deg_84.6805deg_run925___cta-prod4-lst_desert-2180m-LaPalma-lst.simtel.hdf5"
muonDir="/Users/vdk/GeneveWork/Muons/data/"
muonTableName = 'muons/muon_table'


'''Initialize all parameters we might need in this notebook from the hdf5 table 
   I'm really sorry, no way found to combine tables which host data of type 'Table'
   So, we have to read all files in once and store all the information we need in separate np arrays'''

usedParameters = [  'alt' , 'az', 
            'center_fov_lon', 'center_fov_lat',  'radius', 
            'impact_x', 'impact_y', 
            'core_x', 'core_y',
            'energy',
            'optical_efficiency', 
            'width',
            'impact_error', 'phi_error', 'width_error', 'optical_efficiency_error'
         ]

telParameters = [ 'telalt', 'telaz']

'''and the corresponding units'''
usedUnits = { 'alt' : u.deg, 'az': u.deg, 
              'center_fov_lon': u.deg, 'center_fov_lat': u.deg, 'radius' : u.deg, 
              'impact_x' : u.m, 'impact_y' : u.m, 
              'core_x' : u.m, 'core_y' : u.m,
              'energy' : u.TeV, 
              'optical_efficiency' : u.dimensionless_unscaled,
              'width' : u.deg, 
              'impact_error' : u.m, 'phi_error' : u.deg, 'width_error' : u.deg, 
              'optical_efficiency_error' : u.dimensionless_unscaled
            }

telUnits = { 'telalt' : u.deg , 'telaz' : u.deg }

'''Initialize the dictionary for easier access later and loops '''
muonData = {}
for parameterName in usedParameters:
    muonData[parameterName] = np.array([]) 
for telparName in telParameters:
    muonData[telparName] = np.array([]) 

file = muonDir+muonFile
print ('reading ',file)
# tel_pointing_alt = 90. - float(((muonFile.split('_')[1]).split('d'))[0])
# tel_pointing_az = float(((muonFile.split('_')[2]).split('d'))[0])
#telData = { 'telalt' : tel_pointing_alt, 'telaz' : tel_pointing_az }
telData = { 'telalt' : 90., 'telaz' : 0. }
# print ('Telescope points at: ', tel_pointing_alt,'deg Alt and ', tel_pointing_az,'deg Az ')
h5Root  = tb.open_file(file)
print (h5Root)
print (h5Root.root.muons.muon_table)
h5Table = h5Root.root.muons.muon_table
print (h5Table.flush)
for colname in h5Table.colnames: 
    print(f"Column name: {colname}")
    # Optionally, print the first few elements of each column to check
    print(f"First few elements of column '{colname}': {h5Table.col(colname)[:]}")
for parameterName in usedParameters:
    arrayWithUnit = ( np.array(  [ x[parameterName] for x in h5Table.iterrows() ] ) ) * usedUnits[parameterName]
    #print (len(arrayWithUnit))
    muonData[parameterName] = np.concatenate( (muonData[parameterName], arrayWithUnit) )
for telparName in telParameters: 
    #print (len(h5Table), telData[telparName])
    arrayWithUnit = ( np.ones(len(h5Table))  * telData[telparName] ) #* telUnits[telparName]
    muonData[telparName] = np.concatenate( (muonData[telparName], arrayWithUnit) )
print (h5Table.iterrows())
print('\nDone with reading all {:d} muon data arrays from the hdf5 file into memory'.format(len(usedParameters)))

In [ ]:

from astropy.coordinates import SkyCoord, AltAz
from ctapipe.coordinates import NominalFrame


In [ ]:
"""These values apply only to FlashCam !!!"""
camera_rotation = 0.* u.deg # rotation seems to be correced with current version, previously 30.*u.deg
"""For Nectar Cam and LST Cam use:
camera_rotation = 100.893 * u.deg """

In [ ]:
telescope_alt =  u.Quantity(muonData['telalt'],u.deg).to(u.rad) #1.5707963  * u.rad   # If muons have been simulated with telescope pointing to zenith 
telescope_az  =  u.Quantity(muonData['telaz'],u.deg).to(u.rad) #- 0.09284279 * u.rad   # Magnetic North, used for simulations with Corsika 

""" The next lines are only to avoid warnings from ctapipe.coordinates and are not needed for MC """
from astropy.time import Time
from astropy.coordinates import EarthLocation
""" Adopt a dummy time and an earth location"""
obstime = Time('2013-11-01T03:00')
location = EarthLocation.of_site('Roque de los Muchachos')
altaz = AltAz(location=location, obstime=obstime)

telescope_pointing = SkyCoord(alt=telescope_alt, az=telescope_az, frame=altaz)

In [ ]:
alts = muonData['alt']
"""
ATTENTION !!!! 
HAVE TO SUBTRACT HERE THE CAMERA ROTATION FOR FLASHCAM WHICH WAS NOT CORRECTLY 
TAKEN INTO ACCOUNT DURING THE PRODUCTION OF THE MUON PARAMETERS !!!!
"""
azs  = muonData['az'] - camera_rotation

#print (alts)

"""Convert the original muon directions into the telescope pointing frame """
ring_original       = SkyCoord(az=azs, alt=alts, frame=altaz)
ring_original_sky   = ring_original.transform_to( NominalFrame(origin=telescope_pointing) )

ring_original_sky_x = ring_original_sky.fov_lon.to(u.deg)
ring_original_sky_y = ring_original_sky.fov_lat.to(u.deg)

#print (ring_original_sky_x)
#print (ring_original_sky_y)

In [ ]:
ring_delta_az  = muonData['center_fov_lon']
ring_delta_alt = muonData['center_fov_lat']
radius         = muonData['radius']

In [ ]:
event.simulation.tel[1].true_image